In [1]:
import sys 
sys.path.append('../..')

In [2]:
import zarr 

In [3]:
from main_utils import LLM_EmbedData

In [4]:
data = LLM_EmbedData(
    data_source_yaml_path='../../datasets/product_title_embedding/wish-tahoe-pseudo-test.yaml', 
    model_name='sentence-transformers/sentence-t5-large'
)

In [5]:
data.hparams.raw_cache_dir_folder

'/data/junwang/.cache/general/e2713f4e00f4e84a004ee2968a1d8425'

In [6]:
!ls /data/junwang/.cache/general/e2713f4e00f4e84a004ee2968a1d8425

test.json.gz  test.json.gz.lock


In [7]:
!du -sh /data/junwang/.cache/general/e2713f4e00f4e84a004ee2968a1d8425

6.4G	/data/junwang/.cache/general/e2713f4e00f4e84a004ee2968a1d8425


In [8]:
import pandas as pd
import os

In [9]:
df = pd.read_json(
    os.path.join(data.hparams.raw_cache_dir_folder, 'test.json.gz'), 
    lines=True)

In [10]:
len(df) / 1e6

195.372388

In [11]:
z = zarr.open('../../models/product_title_embedding/version_0/epoch=4-step=370604--wish-tahoe-pseudo-test.zarr', 'r')

In [12]:
list(z.array_keys())

['dl-idx_b-idx_s-idx_pred_rank-0']

In [13]:
import numpy as np

In [14]:
z['dl-idx_b-idx_s-idx_pred_rank-0'].shape[0] / 1e6

77.28

In [15]:
import random

In [16]:
sample_arr = z['dl-idx_b-idx_s-idx_pred_rank-0'].oindex[
    random.sample(range(z['dl-idx_b-idx_s-idx_pred_rank-0'].shape[0]),int(1e6)), :]

In [17]:
s_idx = sample_arr[:,2].astype(int)
embs = sample_arr[:,3:]

In [18]:
s_idx

array([27587945, 13243002, 21798751, ..., 70212466, 68044253, 17219541])

In [19]:
df_sample = df.iloc[s_idx].reset_index()

In [20]:
df_sample

,index,text_input,text_output
0,27587945,Embed product: LIUBETY Decorative Throw Pillow...,Embed taxonomy: bedroom > home decor > pillow ...
1,13243002,Embed product: E1SYNDICATE T-SHIRT MDMA RED DE...,Embed taxonomy: tops > fashion > short sleeves
2,21798751,Embed product: Dead Sea T-Shirt Mens undeadwat...,Embed taxonomy: tops > fashion > shirts
3,23030827,Embed product: Dead Kennedys Portable Bags Clu...,Embed taxonomy: bags > wallets & bags > pouches
4,72772649,Embed product: Omg Santa Santa Coming Pants,Embed taxonomy: fashion > bottoms > shorts
...,...,...,...
999995,51469693,Embed product: WWX01 Dog Paw Prints Pattern De...,Embed taxonomy: accessories > masks
999996,57009459,Embed product: Floral Pattern Unisex Reusable ...,Embed taxonomy: accessories > masks
999997,70212466,Embed product: New Fashion Animal Marlin Hunti...,Embed taxonomy: shirts > tops > fashion > swea...
999998,68044253,Embed product: 2020 NEW Cat 3d T Shirt Show Sh...,Embed taxonomy: tops > fashion > short sleeves


In [21]:
z_tax = zarr.open('../../models/product_title_embedding/version_0/epoch=4-step=370604--wish-newtax-v1.2.1.zarr', 'r')
data_tax = LLM_EmbedData(
    data_source_yaml_path='../../datasets/taxonomy_embedding/wish-newtax-v1.2.1.yaml', 
    model_name='sentence-transformers/sentence-t5-large'
)
df_tax = pd.read_json(
    os.path.join(data_tax.hparams.raw_cache_dir_folder, 'test.json.gz'), 
    lines=True)

In [22]:
df_tax.shape, z_tax['dl-idx_b-idx_s-idx_pred_rank-0'].shape

((6037, 2), (6037, 1027))

In [23]:
arr = z_tax['dl-idx_b-idx_s-idx_pred_rank-0']
s_idx_tax = arr[:,2].astype(int)
embs_tax = arr[:,3:]

In [24]:
df_tax = df_tax.iloc[s_idx_tax].reset_index()

In [25]:
df_tax

,index,text_input,text_output
0,0,Embed taxonomy: security & protection > buildi...,Embed taxonomy: security & protection > buildi...
1,1,Embed taxonomy: security & protection > door i...,Embed taxonomy: security & protection > door i...
2,2,Embed taxonomy: security & protection > emerge...,Embed taxonomy: security & protection > emerge...
3,3,Embed taxonomy: security & protection > fire p...,Embed taxonomy: security & protection > fire p...
4,4,Embed taxonomy: security & protection > iot de...,Embed taxonomy: security & protection > iot de...
...,...,...,...
6032,6032,Embed taxonomy: women's clothing > underwear &...,Embed taxonomy: women's clothing > underwear &...
6033,6033,Embed taxonomy: women's clothing > underwear &...,Embed taxonomy: women's clothing > underwear &...
6034,6034,Embed taxonomy: women's clothing > underwear &...,Embed taxonomy: women's clothing > underwear &...
6035,6035,Embed taxonomy: women's clothing > underwear &...,Embed taxonomy: women's clothing > underwear &...


In [26]:
embs_tax.shape

(6037, 1024)

In [27]:
embs.shape

(1000000, 1024)

In [28]:
embs

array([[ 0.18753853,  0.23982352, -1.93111479, ..., -0.68145633,
         0.84323353, -0.87728482],
       [ 0.10616554,  0.67645431,  0.05028763, ...,  1.20773911,
        -1.23720419, -0.86730665],
       [ 0.03214292,  0.25528508,  0.21895012, ...,  0.33676004,
         0.02206876, -0.06674462],
       ...,
       [ 0.67787844,  0.10636625,  0.09057429, ...,  0.96151429,
         1.03130782, -1.39424753],
       [-0.91273552,  0.00465587, -0.52761531, ...,  1.1218828 ,
         0.22350179, -1.17181718],
       [-1.01307166,  0.33498546, -2.00109291, ..., -1.63484514,
        -1.40571547, -1.13246679]])

In [29]:
import torch
from torch import nn

In [30]:
embs_tax_cuda = torch.tensor(embs_tax).cuda()

In [31]:
embs_tax_cuda = nn.functional.normalize(embs_tax_cuda, dim=1)

In [32]:
embs_tax_cuda

tensor([[-0.0059,  0.0320,  0.0137,  ...,  0.0275, -0.0143, -0.0158],
        [-0.0151,  0.0096,  0.0094,  ...,  0.0405, -0.0124, -0.0372],
        [ 0.0314,  0.0567,  0.0249,  ..., -0.0450,  0.0147, -0.0464],
        ...,
        [ 0.0182,  0.0298,  0.0267,  ...,  0.0452, -0.0207,  0.0038],
        [ 0.0091,  0.0434,  0.0405,  ...,  0.0402, -0.0151, -0.0074],
        [ 0.0058,  0.0561,  0.0471,  ...,  0.0354, -0.0264, -0.0041]],
       device='cuda:0', dtype=torch.float64)

In [33]:
embs_chunks = np.array_split(embs, 100)

In [34]:
from tqdm import tqdm

In [35]:
sim_chunks = []
for chunk in tqdm(embs_chunks):
    chunk_ = nn.functional.normalize(torch.tensor(chunk).cuda(), dim=1)
    sim_chunks.append(chunk_.mm(embs_tax_cuda.T).detach().cpu())

100%|██████████| 100/100 [00:45<00:00,  2.18it/s]


In [36]:
sims = np.vstack(sim_chunks)

In [37]:
sims.shape

(1000000, 6037)

In [38]:
preds = sims.argmax(1)
df_preds = df_tax.iloc[preds]
df_sample['pred'] = df_preds['text_input'].apply(lambda x: x.split('Embed taxonomy: ')[1]).tolist()

In [42]:
df_sample.sample(5).to_dict('records')

[{'index': 63196477,
  'text_input': 'Embed product: Lion Wall Art Canvas Painting Wall Picture Art Print, Lion Posters for Room Modern Home Decor for Living Room(no Frame)',
  'text_output': 'Embed taxonomy: home decor > wall art',
  'pred': 'home & garden > home decor > painting & calligraphy'},
 {'index': 40888127,
  'text_input': 'Embed product: Our favorite Women Charm Lace Lady Briefs Lingerie Knickers G-string Thongs Panties Underwear',
  'text_output': 'Embed taxonomy: fashion > underwear',
  'pred': "women's clothing > women's exotic apparel > panties"},
 {'index': 46818007,
  'text_input': "Embed product: Clerks Berserker Funny Movie T Shirt Summer Men'S fashion Tee,Comfortable t shirt,Casual Short Sleeve TEE hip hop funny tee",
  'text_output': 'Embed taxonomy: tops > fashion > short sleeves',
  'pred': "men's clothing > tops & tees > t-shirts"},
 {'index': 10765811,
  'text_input': 'Embed product: Wholesale Dog Paw Dangle Navel Ring Body Jewelry Piercings 316L Surgical Stee

In [43]:
from collections import Counter

In [44]:
len(df_sample)

1000000

In [45]:
Counter(df_sample['pred'].apply(lambda x: x.split(' > ')[0])).most_common()

[('home & garden', 229951),
 ('sports', 162343),
 ("men's clothing", 119744),
 ('jewelry & accessories', 75139),
 ("women's clothing", 67025),
 ('education & office supplies', 38512),
 ('automobiles & motorcycles', 37855),
 ('mother & kids', 35471),
 ('apparel accessories', 30722),
 ('home improvement', 27902),
 ('luggage & bags', 25275),
 ('beauty & health', 19959),
 ('toys & hobbies', 19945),
 ('cellphones & telecommunications', 18576),
 ('novelty & special use', 13647),
 ('consumer electronics', 11902),
 ('virtual goods', 11669),
 ('computer & office', 11212),
 ('security & protection', 8830),
 ('shoes', 7273),
 ('tools', 7109),
 ('watches', 4956),
 ('entertainment', 4897),
 ('furniture', 4014),
 ('food', 3486),
 ('hair extensions & wigs', 1831),
 ('electronic components & supplies', 755)]

In [98]:
229294 / 803

285.546699875467

In [100]:
df_sample[df_sample['pred'].apply(lambda x: "electronic components & supplies >" in x)].sample(5).to_dict('records')

[{'index': 53306093,
  'text_input': 'Embed product: tiki bar open tin metal sign garage man cave plans',
  'text_output': 'Embed taxonomy: home decor > signs',
  'pred': 'electronic components & supplies > electronic signs'},
 {'index': 64942003,
  'text_input': 'Embed product: SLIMRUN  CAT6 UTP CABLE-5FT BLUE',
  'text_output': 'Embed taxonomy: gadgets > cables',
  'pred': 'electronic components & supplies > electronic data systems'},
 {'index': 60252493,
  'text_input': 'Embed product: bedroom ideas Western arms and ammuntion Super-X for long range power tin sign',
  'text_output': 'Embed taxonomy: home decor > wall art',
  'pred': 'electronic components & supplies > electronic signs'},
 {'index': 1285639,
  'text_input': 'Embed product: 10Pcs 20Pcs 50Pcs 100Pcs 20cm 3mm/5mm LED Lamp Cable Bulb Pre-wired DC Emitting Diode Light',
  'text_output': 'Embed taxonomy: gadgets > led lights',
  'pred': 'electronic components & supplies > active components > diodes'},
 {'index': 15818021,
 

In [47]:
df_sample[df_sample['pred'].apply(lambda x: "home & garden > festive & party supplies > wedding decorations > aisle runners" in x)].sample(5).to_dict('records')

[{'index': 54401824,
  'text_input': 'Embed product: 2 Pieces Soft Kitchen Rugs, Koi Fish Hibiscus Flower Pattern by Hand Art Highly Detailed in Line Patterns Batik Paint Washable Long Kitchen Mat Set 15.7" x 23.6"+15.7" x 47.2" Rugs for Kitchen Floor',
  'text_output': 'Embed taxonomy: living room > home decor > carpets',
  'pred': 'home & garden > festive & party supplies > wedding decorations > aisle runners'},
 {'index': 64693297,
  'text_input': 'Embed product: Dual Rib Indoor Mat&#44; Vinyl Backing&#44; 3 in. x 5 in.&#44; Chocolate',
  'text_output': 'Embed taxonomy: home decor > living room',
  'pred': 'home & garden > festive & party supplies > wedding decorations > aisle runners'},
 {'index': 13441012,
  'text_input': 'Embed product: St Patrick\'s Day Decoration Doormat Home Decor Classic Plaid Stripes Clover Welcome Indoor Outdoor Entrance Bathroom Floor Mats Non Slip Washable Hoilday Pet Food Mat, 20" X 30"Inch',
  'text_output': 'Embed taxonomy: living room > home decor > c

In [48]:
df_openai_reverse = pd.read_json('../../models/product_title_embedding/version_0/epoch=4-step=370604--wish-tahoe-openai-reversegen.json', lines=True)

In [50]:
tmp = LLM_EmbedData('../../datasets/product_title_embedding/wish-tahoe-openai-reversegen.yaml')
tmp.prepare_data()

/opt/conda/envs/py38/lib/python3.8/site-packages/transformers/models/t5/tokenization_t5_fast.py:156: FutureWarning: This tokenizer was incorrectly instantiated with a model max length of 512 which will be corrected in Transformers v5.
For now, this behavior is kept to avoid breaking backwards compatibility when padding/encoding with `truncation is True`.
- Be aware that you SHOULD NOT rely on t5-base automatically truncating your input to 512 when padding/encoding.
- If you want to encode/pad to sequences longer than 512 you can either instantiate this tokenizer with `model_max_length` or pass `max_length` when encoding/padding.
- To avoid this warning, please instantiate this tokenizer with `model_max_length` set to your preferred value.
  warnings.warn(


In [52]:
ds_openai_reverse = tmp.get_hf_dataset()

In [53]:
ds_openai_reverse

DatasetDict({
    test: Dataset({
        features: ['text_input', 'text_output'],
        num_rows: 128659
    })
    train: Dataset({
        features: ['text_input', 'text_output'],
        num_rows: 145399
    })
    val: Dataset({
        features: ['text_input', 'text_output'],
        num_rows: 122960
    })
})

In [55]:
tmp = pd.DataFrame(ds_openai_reverse['test'][:len(df_openai_reverse)])

In [56]:
tmp

,text_input,text_output
0,Embed product: Buckle Down Seatbelt Belt boys ...,Embed taxonomy: apparel accessories > boys' ac...
1,Embed product: Boys' Blue Light Blocking Glass...,Embed taxonomy: apparel accessories > boys' ac...
2,Embed product: Boys' Stylish Rectangular Plast...,Embed taxonomy: apparel accessories > boys' ac...
3,Embed product: Sunglasses for Boys - Fashionab...,Embed taxonomy: apparel accessories > boys' ac...
4,Embed product: Warm Winter Gloves for Boys - E...,Embed taxonomy: apparel accessories > boys' ac...
...,...,...
98197,Embed product: Professional Stainless Steel M...,Embed taxonomy: beauty & health > makeup > mak...
98198,Embed product: Premium Makeup Tool Kit - 12 Pi...,Embed taxonomy: beauty & health > makeup > mak...
98199,Embed product: Hair Loss Treatment Men's Shamp...,Embed taxonomy: beauty & health > men's groomi...
98200,Embed product: Jack Black Double-Duty Face Was...,Embed taxonomy: beauty & health > men's groomi...


In [57]:
df_openai_reverse['title'] = tmp['text_input'].apply(lambda x: x.split('Embed product: ')[1])
df_openai_reverse['taxonomy'] = tmp['text_output'].apply(lambda x: x.split('Embed taxonomy: ')[1])

In [60]:
df_openai_reverse_agg = df_openai_reverse.groupby('taxonomy').agg(
    {"embedding": lambda x: np.array([i for i in x]).mean(0).tolist()}
)

In [65]:
df_openai_reverse_agg = df_openai_reverse_agg.reset_index()

In [63]:
df_tax['taxonomy'] = df_tax['text_input'].apply(lambda x: x.split('Embed taxonomy: ')[1])

In [66]:
df_openai_reverse_agg

,taxonomy,embedding
0,apparel accessories > boys' accessories > boys...,"[0.33757509915158124, 2.0586912453174593, 1.04..."
1,apparel accessories > boys' accessories > boys...,"[0.5721632391796445, 0.5484681069850919, 0.070..."
2,apparel accessories > boys' accessories > boys...,"[-0.2087530322372912, 0.42678223438560925, 0.1..."
3,apparel accessories > boys' accessories > boys...,"[-0.2707704186439512, 0.07436672411859022, -0...."
4,apparel accessories > boys' accessories > boys...,"[-1.497097337245941, -0.24853019751608355, 0.1..."
...,...,...
5286,women's clothing > women's exotic apparel > li...,"[0.3948894550816878, -0.12318266379750413, 0.1..."
5287,women's clothing > women's exotic apparel > pa...,"[1.1524825676211286, 0.034289209172129555, 0.3..."
5288,women's clothing > women's exotic apparel > pants,"[0.7438778032859165, 0.12014147535794296, 0.43..."
5289,women's clothing > women's exotic apparel > tanks,"[0.44326331721324624, 0.3391908056249745, 1.16..."


In [67]:
df_tax_merge = df_tax.merge(df_openai_reverse_agg, on='taxonomy', how='inner')

In [69]:
embs_tax_new = np.array(df_tax_merge.embedding.tolist())

In [71]:
embs_tax_new_cuda = nn.functional.normalize(torch.tensor(embs_tax_new).cuda(), dim=1)

In [72]:
sim_chunks_new = []
for chunk in tqdm(embs_chunks):
    chunk_ = nn.functional.normalize(torch.tensor(chunk).cuda(), dim=1)
    sim_chunks_new.append(chunk_.mm(embs_tax_new_cuda.T).detach().cpu())
sims_new = np.vstack(sim_chunks_new)
preds_new = sims_new.argmax(1)
df_preds_new = df_tax_merge.iloc[preds_new]


100%|██████████| 100/100 [00:33<00:00,  2.96it/s]


In [74]:
df_sample['pred_new'] = df_preds_new['text_input'].apply(lambda x: x.split('Embed taxonomy: ')[1]).tolist()

In [78]:
len(df_sample), len(df_sample[df_sample.pred == df_sample.pred_new])

(1000000, 374616)

In [79]:
Counter(df_sample[df_sample.pred == df_sample.pred_new]['pred'].apply(lambda x: x.split(' > ')[0])).most_common()

[("men's clothing", 99044),
 ('home & garden', 95654),
 ("women's clothing", 35280),
 ('sports', 25125),
 ('jewelry & accessories', 22926),
 ('automobiles & motorcycles', 16390),
 ('luggage & bags', 11403),
 ('mother & kids', 10042),
 ('apparel accessories', 8605),
 ('beauty & health', 6928),
 ('education & office supplies', 6887),
 ('home improvement', 5397),
 ('computer & office', 5048),
 ('cellphones & telecommunications', 4728),
 ('consumer electronics', 4053),
 ('toys & hobbies', 3468),
 ('shoes', 3179),
 ('tools', 3087),
 ('novelty & special use', 2200),
 ('entertainment', 1468),
 ('furniture', 1019),
 ('virtual goods', 846),
 ('watches', 705),
 ('security & protection', 627),
 ('electronic components & supplies', 298),
 ('hair extensions & wigs', 132),
 ('food', 77)]

In [81]:
df_sample[df_sample.pred == df_sample.pred_new].sample(3)[['text_input', 'pred', 'pred_new']].to_dict('records')

[{'text_input': "Embed product: tobiijhleget Work Sucks Let's Ride Cap Custom Pattern Baseball Cap The New Printed Fashion Cap Hat Adjustable Adult Sun Casquette",
  'pred': "apparel accessories > women's hats > women's baseball caps",
  'pred_new': "apparel accessories > women's hats > women's baseball caps"},
 {'text_input': 'Embed product: 2020 Fashion T-shirt. Was Not Me Saying Funny Mens T-shirt, Joke Graphic Design Printed Tee',
  'pred': "men's clothing > tops & tees > t-shirts",
  'pred_new': "men's clothing > tops & tees > t-shirts"},
 {'text_input': 'Embed product: SignMission SSA-A-17 Nomad 4 x 18 in. Air Force Aircraft Military Street Sign - A-17 Nomad',
  'pred': 'electronic components & supplies > electronic signs',
  'pred_new': 'electronic components & supplies > electronic signs'}]